In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

correct_squat_1_path = "Clean_Dataset/Correct_Squat_1.xlsx"
correct_squat_2_path = "Clean_Dataset/Correct_Squat_2.xlsx"
incorrect_squat_1_path = "Clean_Dataset/Incorrect_Squat_1.xlsx"
incorrect_squat_2_path = "Clean_Dataset/Incorrect_Squat_2.xlsx"

correct_squat_1 = pd.read_excel(correct_squat_1_path)
correct_squat_2 = pd.read_excel(correct_squat_2_path)
incorrect_squat_1 = pd.read_excel(incorrect_squat_1_path)
incorrect_squat_2 = pd.read_excel(incorrect_squat_2_path)

correct_squat_1['Label'] = 1
correct_squat_2['Label'] = 1
incorrect_squat_1['Label'] = 0
incorrect_squat_2['Label'] = 0

all_data = pd.concat([correct_squat_1, correct_squat_2, incorrect_squat_1, incorrect_squat_2], ignore_index=True)

all_data.shape, all_data.head()


((1354, 9),
    Rep  Time (s)  Rotation Rate x  Rotation Rate y  Rotation Rate z  \
 0    0  0.046980        -0.171921        -0.000695        -0.014218   
 1    0  0.147845        -0.024179         0.064703         0.009083   
 2    0  0.248211        -0.024391         0.102901         0.054025   
 3    0  0.348784        -0.120931         0.135223         0.051266   
 4    0  0.449145        -0.267242         0.275167        -0.000876   
 
    Acceleration x  Acceleration y  Acceleration z  Label  
 0        0.031045       -0.038933        0.031179      1  
 1        0.004461        0.007454       -0.009542      1  
 2        0.033564        0.007207       -0.026100      1  
 3        0.061731       -0.016650        0.033037      1  
 4        0.124680       -0.036982        0.062302      1  )

##### Data Processing 

In [3]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

features = all_data.drop('Label', axis=1)
labels = all_data['Label']

scaler = StandardScaler()
features_normalized = scaler.fit_transform(features)

X_train, X_test, y_train, y_test = train_test_split(features_normalized, labels, test_size=0.2, random_state=42)

##### Define and Train the Neural Network Models

In [4]:
from keras.models import Sequential
from keras.layers import Dense

def create_model(input_shape):
    model = Sequential([
        Dense(64, activation='relu', input_shape=(input_shape,)),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = create_model(X_train.shape[1])
model.fit(X_train, y_train, epochs=10, validation_split=0.1)


2024-02-23 12:54:09.723588: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/data_adapter.py:1808: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  return t[start:end]


31/31 [==============================] - 1s 11ms/step - loss: 0.6686 - accuracy: 0.5616 - val_loss: 0.6570 - val_accuracy: 0.5596
Epoch 2/10
31/31 [==============================] - 0s 3ms/step - loss: 0.6329 - accuracy: 0.6201 - val_loss: 0.6249 - val_accuracy: 0.5872
Epoch 3/10
31/31 [==============================] - 0s 3ms/step - loss: 0.5990 - accuracy: 0.6684 - val_loss: 0.5863 - val_accuracy: 0.6972
Epoch 4/10
31/31 [==============================] - 0s 4ms/step - loss: 0.5588 - accuracy: 0.7464 - val_loss: 0.5362 - val_accuracy: 0.7798
Epoch 5/10
31/31 [==============================] - 0s 3ms/step - loss: 0.5139 - accuracy: 0.8049 - val_loss: 0.4771 - val_accuracy: 0.8807
Epoch 6/10
31/31 [==============================] - 0s 3ms/step - loss: 0.4655 - accuracy: 0.8429 - val_loss: 0.4207 - val_accuracy: 0.9083
Epoch 7/10
31/31 [==============================] - 0s 3ms/step - loss: 0.4189 - accuracy: 0.8552 - val_loss: 0.3669 - val_accuracy: 0.8991
Epoch 8/10
31/31 [============

##### Printing performance

In [5]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

9/9 [==============================] - 0s 2ms/step - loss: 0.3126 - accuracy: 0.9114
Test Accuracy: 0.9114391207695007


In [15]:
from keras.models import load_model
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Load the test data
test_data_path = "Clean_Dataset/correct form testing data.xls"
test_data = pd.read_excel(test_data_path, engine='xlrd')
test_data = test_data.drop(columns=['Unnamed: 0'], errors='ignore')

# Assuming 'scaler' is already fitted to your training data
# Scale the features directly since test_data now only contains features
raw_data_scaled = scaler.transform(test_data)

# Load your trained model
model = load_model('model.h5')

# Make predictions with the scaled data
prediction = model.predict(raw_data_scaled)
predicted_label = (prediction > 0.5).astype(int)  # Assuming binary classification with a threshold of 0.5

# Print the prediction result
if predicted_label[0] == 1:
    print("The squat set is correct.")
else:
    print("The squat set needs adjustment.")




ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Time frame
- acceleration_x
- acceleration_y
- acceleration_z
- rotation_x
- ...
Feature names seen at fit time, yet now missing:
- Acceleration x
- Acceleration y
- Acceleration z
- Rep
- Rotation Rate x
- ...
